In [2]:
import csv
from pathlib import Path

import censusgeocode as cg
import duckdb
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
DATA_DIR = Path.cwd().parent.parent / "data"
GEOCODED_DIR = DATA_DIR / "geocoded"

GEOCODED_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
conn = duckdb.connect(str(DATA_DIR / "lihtc.duckdb"))
df = conn.query("SELECT * FROM lihtc;").to_df()
conn.close()

In [5]:
df.head()

,hud_id,project,proj_add,proj_cty,proj_st,proj_zip,state_id,latitude,longitude,place1990,...,n_unitsr,li_unitr,metro,dda,qct,nonprog,nlm_reason,nlm_spc,datanote,record_stat
0,AKA0000X003,EAGLE RIDGE TOWNHOMES,1775 NORTH THUMA,PALMER,AK,99645,AK-05-23,61.587872,-149.099777,NaN,...,33.0,33.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,X
1,AKA0000X018,"GATEWAY-SEWARD ASSOCIATES, LTD PTN",1810 PHOENIX ROAD,SEWARD,AK,99664,AK-99-99,60.125469,-149.446060,NaN,...,20.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
2,AKA0000X022,"JUNEAU AFFORDABLE RENTALS, LLC",SCATTERED SITE,JUNEAU,AK,99801,AK-99-99,58.383499,-134.437698,NaN,...,25.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
3,AKA0000X024,"MILL BAY TOWNHOMES, LLC",1223 MILL BAY ROAD,KODIAK,AK,99615,AK-99-99,57.796478,-152.392319,NaN,...,20.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
4,AKA0000X030,TURNAGAIN PLACE APTS,2708 COHO WAY,ANCHORAGE,AK,99517,AK-99-99,61.205101,-149.930298,NaN,...,29.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X


In [6]:
df["my_id"] = np.arange(len(df), dtype=int)
data = df[["my_id", "proj_add", "proj_cty", "proj_st", "proj_zip"]]

In [7]:
# Some decent amount of missing data
# ZIP code doesn't really matter (I think), so let's just drop missing address and cty
data.isna().sum()

my_id          0
proj_add     987
proj_cty      42
proj_st        0
proj_zip    3106
dtype: int64

In [8]:
data2 = data[data["proj_add"].notna() & data["proj_cty"].notna()]

In [9]:
# Keep to 5-digit ZIP codes
data2["my_zip"] = data2["proj_zip"].str[:5]
data2.drop(columns="proj_zip", inplace=True)

/var/folders/_8/wxdcgmrd7b931622mzvpxn6c0000gq/T/ipykernel_73215/653356926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2["my_zip"] = data2["proj_zip"].str[:5]
/var/folders/_8/wxdcgmrd7b931622mzvpxn6c0000gq/T/ipykernel_73215/653356926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.drop(columns="proj_zip", inplace=True)


In [10]:
import io

def get_io(df) -> io.StringIO:
    s = io.StringIO()
    writer = csv.writer(s)
    writer.writerows(df.values)
    s.seek(0)
    return s

s = get_io(data2)

In [11]:
# Make sure that looks right
s.seek(0)
print(s.read(1_000))

0,1775 NORTH THUMA,PALMER,AK,99645
1,1810 PHOENIX ROAD,SEWARD,AK,99664
2,SCATTERED SITE,JUNEAU,AK,99801
3,1223 MILL BAY ROAD,KODIAK,AK,99615
4,2708 COHO WAY,ANCHORAGE,AK,99517
5,1250 N LUCILLE ST,WASILLA,AK,99654
6,1240 N LUCILLE ST,WASILLA,AK,99654
7,402-451 NORTH YENLO STREET,WASILLA,AK,99654
8,41718 CRESTED CRANE ST,HOMER,AK,99603
9,660 SOUTH YORKTOWN,KETCHIKAN,AK,99901
10,2012 SANDVIK ST,FAIRBANKS,AK,99709
11,103 BURKHART DR,SITKA,AK,99835
12,190 PARKWOOD CIR,SOLDOTNA,AK,99669
14,1200 S CENTURY CIR,WASILLA,AK,99654
15,231 GASTINEAU AVE,JUNEAU,AK,99801
16,10601 SPRUCE VIEW LOOP,ANCHORAGE,AK,99507
17,429 D ST,ANCHORAGE,AK,99501
18,925 SOUTH CHUGACH,PALMER,AK,99645
19,429 D ST.,ANCHORAGE,AK,99501
20,1300 N YORKTOWN DR,KETCHIKAN,AK,99901
21,HAMPSTEAD CT,ANCHORAGE,AK,99518
22,1801 DOUGLAS HWY,JUNEAU,AK,99824
23,9200 INDEPENDENCE DR,ANCHORAGE,AK,99507
24,401 ORCA AVE,CORDOVA,AK,99574
25,310 W 76TH AVE,ANCHORAGE,AK,99518
26,315 BARROW ST,ANCORAGE,AK,99501
27,301 


In [11]:
# It does, so let's loop!
step_size = 1_000
output = []
for i in tqdm(range(0, len(data2), step_size)):
    subset = data2.iloc[i:i+step_size]
    s = get_io(subset)
    blah = cg.addressbatch(s)
    output.append(blah)

100%|██████████| 50/50 [37:11<00:00, 44.63s/it]


In [14]:
geocoded_df = pd.concat(map(pd.DataFrame, output))

In [19]:
geocoded_df["id"] = geocoded_df["id"].astype(int)

In [1]:
merged_df = df.merge(
    geocoded_df,
    how="left",
    left_on="my_id",
    right_on="id",
)

NameError: name 'df' is not defined

In [20]:
merged_df.to_parquet(GEOCODED_DIR / "census_geocoder.parquet")